In [11]:
# Import stuff
from pandas import DataFrame

from nipype.pipeline.engine import Workflow, Node, MapNode, JoinNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import DataSink, FreeSurferSource, SelectFiles
from nipype.algorithms.misc import Gunzip

from nipype.interfaces.freesurfer.preprocess import MRIConvert
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer import FSCommand, MRIConvert, ReconAll
from nipype.interfaces.fsl.utils import Reorient2Std, MotionOutliers, Merge
from nipype.interfaces.fsl.preprocess import MCFLIRT, SliceTimer, FLIRT
from nipype.interfaces.fsl.maths import ApplyMask
from nipype.interfaces.fsl.model import GLM
from nipype.interfaces.fsl.epi import ApplyTOPUP, TOPUP
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.nipy.preprocess import Trim

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/home/camachocm2/Analysis/ChEC/fmri_proc'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
secondlevel_dir = analysis_home + '/grouplevel'
workflow_dir = analysis_home + '/workflows'
phase_encoding_file = analysis_home + '/misc/chec_encoding_file.txt'
mni_template = analysis_home + '/template/MNI152_T1_3mm_brain.nii.gz'
mni_brainmask = analysis_home + '/template/MNI152_T1_3mm_brain_mask.nii.gz'

#subject_info = DataFrame.read_csv(analysis_home + '/../misc/subjs.csv')
#subjects_list = subject_info['SubjID'].tolist()
subjects_list = ['1000','1001','1002','1006','1007','1008','1011','1015','1016','1017']

# FreeSurfer set up - change SUBJECTS_DIR 
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

# data collection specs
TR = 0.8 #in seconds
num_slices = 40
slice_direction = False #True = z direction, top to bottom
interleaved = True
echo_train_length = 0.068 # in seconds, loc (0x18,0x91) in the dcm header
TEs = ['13','38','64']

In [12]:
## Data handling Nodes

# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# Sink data
substitutions = [('_subjid_', ''),
                 ('_func_','')] #output file name substitutions
datasink = Node(DataSink(base_directory = preproc_dir,
                        container = preproc_dir,
                        substitutions = substitutions), 
                name='datasink')

## fMRI data Reconstruction Workflow

In [ ]:
# Pull files
rawfile_template = {'folder': raw_dir + '/{subjid}/{func}'}
recon_selectfiles = Node(SelectFiles(rawfile_template), name='recon_selectfiles')
recon_selectfiles.iterables = [('func',['chec_movie_PA','corr_chec_1','corr_chec_2'])]

In [ ]:
## reconstruction nodes

def sort_dicoms(dicom_directory):    
    from pydicom import dcmread
    from pandas import DataFrame
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    from os.path import isdir
    from os import makedirs
    import shutil
    
    dicoms = glob(dicom_directory + '/MR.*')
    dicoms = sorted(dicoms)
     
    for dicom in dicoms:
        info = dcmread(dicom)
        te = info[0x18,0x81].value
        
        if not isdir(dicom_directory + '/TE_%d' % te):
                makedirs(dicom_directory + '/TE_%d' % te)

        shutil.move(dicom, dicom_directory + '/TE_%d' % te)
    
    dicom_names = []
    TE_folders = glob(dicom_directory + '/TE_*')
    TE_folders = sorted(TE_folders)
    for folder in TE_folders:
        temp = glob(folder + '/MR*')
        temp = sorted(temp)
        dicom_names.append(temp[0])
        
    return(dicom_names)

In [ ]:
# Pull Dicom information
dicom_info = Node(Function(input_names=['dicom_directory'],
                           output_names=['dicom_names'],  
                           function=sort_dicoms), 
                       name='main_dicom_info')

# convert to niftis
convert_to_niftis = MapNode(MRIConvert(out_file='func.nii.gz', 
                                       out_type='niigz'), 
                            name='convert_to_niftis', 
                            iterfield=['in_file'])

In [ ]:
recon_flow = Workflow(name='recon_flow')
recon_flow.connect([(infosource, recon_selectfiles,[('subjid','subjid')]), 
                    (recon_selectfiles, dicom_info,[('folder','dicom_directory')]), 
                    (dicom_info, convert_to_niftis,[('dicom_names','in_file')]),
                    
                    (convert_to_niftis, datasink, [('out_file','separated_niftis')])
                   ])

recon_flow.base_dir = workflow_dir
recon_flow.write_graph(graph2use='flat')
recon_flow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

## fMRI Data Preprocessing Workflows

In [ ]:
## File handling nodes

pes_template={'PE_vol': preproc_dir + '/separated_niftis/{subjid}/corr_{pe_dir}/_convert_to_niftis0/func.nii.gz'}
pes_selectfiles = Node(SelectFiles(pes_template), name='pes_selectfiles')
pes_selectfiles.iterables = ('pe_dir',['chec_1','chec_2'])

funcs_template={'func': preproc_dir + '/separated_niftis/{subjid}/chec_movie_PA/_convert_to_{func_te}/func.nii.gz'}
funcs_selectfiles = Node(SelectFiles(funcs_template), name='funcs_selectfiles')
funcs_selectfiles.iterables = ('func_te',['niftis0','niftis1','niftis2'])

In [ ]:
# Apply ME-ICA (Kundu, et al., 2011)
def tedana_clean(TE_niftis, TEs):
    from pandas import DataFrame
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from tedana.workflows import tedana_workflow
    
    files = tedana_workflow(TE_niftis, TEs)
    
    
    return(cleaned_data)

In [ ]:
from tedana.workflows import tedana_workflow
from tedana import (decomposition, model, selection, utils)
te1 = '/home/camachocm2/Analysis/ChEC/fmri_proc/preproc/unwarped_funcs/1003/_func_te_niftis0/func_unwarped.nii.gz'
te2 = '/home/camachocm2/Analysis/ChEC/fmri_proc/preproc/unwarped_funcs/1003/_func_te_niftis1/func_unwarped.nii.gz'
te3 = '/home/camachocm2/Analysis/ChEC/fmri_proc/preproc/unwarped_funcs/1003/_func_te_niftis2/func_unwarped.nii.gz'
TEs = [13,38,64]
TE_niftis = [te1,te2,te3]
files = tedana_workflow(TE_niftis, TEs)

print(files)

In [ ]:
## Unwarping and ME cleaning nodes

# include only the first volume of each PE volume
trim_PEs = Node(Trim(end_index=1),name='trim_PEs')

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = JoinNode(Merge(dimension='t',
                           merged_file='merged_pes.nii.gz'), 
                     name='merge_pes', 
                     joinsource='pes_selectfiles', 
                     joinfield='in_files')

topup = Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[2], encoding_file=phase_encoding_file, 
                              method='jac', out_corrected='func_unwarped.nii.gz'), 
                   name='apply_topup')

In [ ]:
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,pes_selectfiles, [('subjid','subjid')]),
                        (infosource,funcs_selectfiles, [('subjid','subjid')]),
                        (pes_selectfiles, trim_PEs, [('PE_vol','in_file')]), 
                        (trim_PEs, merge_pes, [('out_file','in_files')]), 
                        (merge_pes, topup, [('merged_file','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (funcs_selectfiles, apply_topup, [('func','in_files')]),
                        (apply_topup, datasink, [('out_corrected','unwarped_funcs')])
                       ])

prepreprocflow.base_dir = workflow_dir
prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

In [13]:
#file handling
preproc_template={'func': preproc_dir + '/unwarped_funcs/{subjid}/te_niftis1/func_unwarped.nii.gz'}
preproc_selectfiles = Node(SelectFiles(preproc_template), name='preproc_selectfiles')

fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

In [14]:
def fsid_convert(subject_id):
    fs_subject = 'C' + str(subject_id)
    return(fs_subject)

fsid = Node(Function(input_names=['subject_id'], 
                     output_names=['fs_subject'], 
                     function=fsid_convert), 
            name='fsid')

In [15]:
## fMRI Data processing nodes

# reorient images to MNI space standard
reorientFunc = Node(Reorient2Std(terminal_output='file'),
                   name='reorientFunc')

# perform slice time correction
slicetime = Node(SliceTimer(index_dir=slice_direction,
                           interleaved=interleaved,
                           time_repetition=TR),
                name='slicetime')

# realignment using mcflirt
realignmc = Node(MCFLIRT(save_plots=True),
                name='realignmc')

# Coregistration using flirt
coregflt = Node(FLIRT(),
               name='coregflt')
coregflt2 = Node(FLIRT(apply_xfm=True, 
                       out_file='preproc_func.nii.gz'),
                name='coregflt2')

#unzip file before feeding into ART
gunzip_mask = Node(Gunzip(), name='gunzip_mask')
gunzip_func = Node(Gunzip(), name='gunzip_func')

# Artifact detection for scrubbing/motion assessment
art = Node(ArtifactDetect(mask_type='file',
                          parameter_source='FSL',
                          bound_by_brainmask=True,
                          norm_threshold=1,
                          zintensity_threshold=3,
                          use_differences=[True, False]),
           name='art')

# Register to MNI template
reg2mni = Node(FLIRT(reference=mni_template),
               name='reg2mni')
reg2mni2 = Node(FLIRT(apply_xfm=True, 
                      reference=mni_template,
                      out_file='proc_func.nii.gz'),
                name='reg2mni2')

In [16]:
## Anatomical processing
# Convert skullstripped brain to nii, resample to 2mm^3
resample = Node(MRIConvert(out_type='niigz',
                          vox_size=(3,3,3)),
               name='resample')

# Reorient anat to MNI space
reorientAnat = Node(Reorient2Std(terminal_output='file'),
                   name='reorientAnat')

# Create binary mask of resampled, skullstripped anat, dilate, and erode to fill gaps
binarize = Node(Binarize(dilate=1,
                        erode=1,
                        invert=False,
                        min=1,
                        max=255),
               name='binarize')

In [17]:
# Data QC nodes
def create_coreg_plot(epi,anat):
    import os
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epi, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = os.path.abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    from nipype.interfaces.nipy.preprocess import Trim
    
    trim = Trim()
    trim.inputs.in_file = epi
    trim.inputs.end_index = 1
    trim.inputs.out_file = 'epi_vol1.nii.gz'
    trim.run()
    epi_vol = abspath('epi_vol1.nii.gz')
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epi_vol, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename)
    display.close()
    maskcheck_file = abspath(maskcheck_filename)

    return(maskcheck_file)

make_coreg_img = Node(name='make_coreg_img',
                      interface=Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot))

make_checkmask_img = Node(name='make_checkmask_img',
                      interface=Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage))
make_checkmask_img.inputs.brainmask=mni_brainmask

In [18]:
preprocflow = Workflow(name='preprocflow')

preprocflow.connect([(infosource, preproc_selectfiles, [('subjid','subjid')]),
                     (preproc_selectfiles,reorientFunc, [('func','in_file')]),
                     (infosource, fsid, [('subjid','subject_id')]),
                     (fsid, fssource, [('fs_subject','subject_id')]),
                     (fssource, resample, [('brainmask','in_file')]),
                     (resample, reorientAnat, [('out_file','in_file')]),
                     (reorientAnat, binarize, [('out_file','in_file')]),
                     (reorientFunc, slicetime, [('out_file','in_file')]),
                     (slicetime, realignmc, [('slice_time_corrected_file','in_file')]),
                     (reorientAnat, coregflt, [('out_file','reference')]),
                     (realignmc, coregflt, [('out_file','in_file')]),
                     (realignmc, coregflt2, [('out_file','in_file')]),
                     (coregflt, coregflt2, [('out_matrix_file','in_matrix_file')]),
                     (reorientAnat, coregflt2, [('out_file','reference')]),              
                     (coregflt, reg2mni, [('out_file','in_file')]),
                     (coregflt2, reg2mni2, [('out_file','in_file')]),
                     (reg2mni, reg2mni2, [('out_matrix_file','in_matrix_file')]),                
                     
                     (binarize, gunzip_mask,[('binary_file','in_file')]), 
                     (gunzip_mask, art, [('out_file','mask_file')]),
                     (coregflt2, gunzip_func, [('out_file','in_file')]),
                     (gunzip_func, art, [('out_file','realigned_files')]),
                     (realignmc, art, [('par_file','realignment_parameters')]),
                     (coregflt, make_coreg_img, [('out_file','epi')]),
                     (reorientAnat, make_coreg_img, [('out_file','anat')]),
                     (reg2mni2, make_checkmask_img, [('out_file','epi')]),
                     
                     (reg2mni, datasink, [('out_file','reoriented_anat')]),
                     (make_checkmask_img, datasink, [('maskcheck_file','checkmask_image')]),
                     (make_coreg_img, datasink, [('coreg_file','coreg_image')]),
                     (reg2mni2, datasink, [('out_file','mnireg_func')]),
                     (art, datasink, [('plot_files','art_plot')]), 
                     (art, datasink, [('outlier_files','art_outliers')]),
                     (realignmc, datasink, [('par_file','mcflirt_displacement')])
                    ])
preprocflow.base_dir = workflow_dir
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 1, 'memory_gb':8})

180920-12:37:48,679 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/graph.png (graph2use=flat, simple_form=True).
180920-12:37:48,707 workflow INFO:
	 Workflow preprocflow settings: ['check', 'execution', 'logging', 'monitoring']
180920-12:37:48,891 workflow INFO:
	 Running in parallel.
180920-12:37:48,902 workflow INFO:
	 [MultiProc] Running 0 tasks, and 20 jobs ready. Free memory (GB): 8.00/8.00, Free processors: 1/1.
180920-12:37:49,4 workflow INFO:
	 [Node] Setting-up "preprocflow.preproc_selectfiles" in "/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1007/preproc_selectfiles".
180920-12:37:49,14 workflow INFO:
	 [Node] Running "preproc_selectfiles" ("nipype.interfaces.io.SelectFiles")
180920-12:37:49,22 workflow INFO:
	 [Node] Finished "preprocflow.preproc_selectfiles".
180920-12:37:50,904 workflow INFO:
	 [Job 0] Completed (preprocflow.preproc_selectfiles).
180920-12:37:50,911 workflow INFO:
	 [Mult

/usr/local/lib/python3.5/dist-packages/nipype/algorithms/rapidart.py:541: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/threading.py", line 882, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()

180920-13:15:18,516 workflow DEBUG:
	 Needed files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/art.preproc_func_outliers.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/plot.preproc_func.png;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/_0x82c4288608e40cae87ff6657581fa494_unfinished.json;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/_inputs.pklz;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/_node.pklz
180920-13:15:18,519 workflow DEBUG:
	 Needed dirs: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/_report
180920-13:15:18,521 workflow DEBUG:
	 Removing files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/norm.preproc_func.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1016/art/disp.preproc_func.nii;/home/camachocm2/Analys

/usr/local/lib/python3.5/dist-packages/nipype/algorithms/rapidart.py:541: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/threading.py", line 882, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()

180920-13:28:10,928 workflow DEBUG:
	 Needed files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/art.preproc_func_outliers.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/plot.preproc_func.png;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/_0x9702f5eba5f5d3a812cf6df54710baf9_unfinished.json;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/_inputs.pklz;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/_node.pklz
180920-13:28:10,930 workflow DEBUG:
	 Needed dirs: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/_report
180920-13:28:10,933 workflow DEBUG:
	 Removing files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/norm.preproc_func.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1000/art/disp.preproc_func.nii;/home/camachocm2/Analys

...

180920-14:21:07,786 workflow DEBUG:
	 Needed files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/art.preproc_func_outliers.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/plot.preproc_func.png;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/_0xbf656038a076a5de3d165e1f4ba42168_unfinished.json;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/_inputs.pklz;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/_node.pklz
180920-14:21:07,789 workflow DEBUG:
	 Needed dirs: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/_report
180920-14:21:07,791 workflow DEBUG:
	 Removing files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/norm.preproc_func.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1008/art/disp.preproc_func.nii;/home/camachocm2/Analys

/usr/local/lib/python3.5/dist-packages/nipype/algorithms/rapidart.py:541: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/threading.py", line 882, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()

180920-14:36:09,792 workflow DEBUG:
	 Needed files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/art.preproc_func_outliers.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/plot.preproc_func.png;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/_0xf454bdd21dfae2be90f58f579f67bb63_unfinished.json;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/_inputs.pklz;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/_node.pklz
180920-14:36:09,795 workflow DEBUG:
	 Needed dirs: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/_report
180920-14:36:09,797 workflow DEBUG:
	 Removing files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/norm.preproc_func.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1006/art/disp.preproc_func.nii;/home/camachocm2/Analys

/usr/local/lib/python3.5/dist-packages/nipype/algorithms/rapidart.py:541: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/threading.py", line 882, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()

180920-15:08:37,434 workflow DEBUG:
	 Needed files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/art.preproc_func_outliers.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/plot.preproc_func.png;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/_0xc51ba2a9a149c9942e467c55031d4285_unfinished.json;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/_inputs.pklz;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/_node.pklz
180920-15:08:37,437 workflow DEBUG:
	 Needed dirs: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/_report
180920-15:08:37,440 workflow DEBUG:
	 Removing files: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/norm.preproc_func.txt;/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/preprocflow/_subjid_1011/art/disp.preproc_func.nii;/home/camachocm2/Analys